In [ ]:
from os import listdir
from os.path import isfile, join
celebrities = ['../input/celeba-dataset/img_align_celeba/img_align_celeba/' + f for f in listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba') if isfile(join('../input/celeba-dataset/img_align_celeba/img_align_celeba', f))]
me = ['../input/pictureme/' + f for f in listdir('../input/pictureme') if isfile(join('../input/pictureme', f))]

onlyfiles = me + celebrities

In [ ]:
onlyfiles[:10]

In [ ]:
#from PIL import Image  
#from skimage import io


#image = Image.open('../input/pictureofme/000000.jpg')  
#image.show()

#import matplotlib.pyplot as plt
#img = io.imread('../input/pictureofme/000000.jpg')
#plt.imshow(img)

In [ ]:
onlyfiles = onlyfiles[:10000]

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
list_img = []
list_label = []
for i,img_info in enumerate(onlyfiles):
    print (i, ' / ', len(onlyfiles))
    img = load_img(img_info, target_size=(224,224,3))
    img_array = img_to_array(img)
    list_img.append(img_array)
    list_label.append(img_info)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
model = InceptionResNetV2(pooling='avg',include_top=False, weights='imagenet', input_shape=(224,224,3))
model.summary()

In [ ]:
import numpy as np
X = np.array(list_img)
X = preprocess_input(X)

In [ ]:
list_finale = []
for index in range(0, len(list_img)):
    print (index, ' / ', len(list_img))
    ret_vec = model.predict(X[index].reshape(1,224,224,3))
    list_finale.append({'vec':ret_vec.reshape(-1),'original':list_img[index], 'path':list_label[index], 'index':index})

In [ ]:
from keras.preprocessing.image import array_to_img

to_find = 0
%matplotlib inline
import matplotlib.pyplot as plt


plt.imshow(array_to_img(list_finale[to_find]['original']))
plt.show()

In [ ]:
from scipy import spatial
cosine_list = []
for index_image,xt in enumerate(list_finale):
    print (index_image, ' / ', len(list_finale))
    result = 1 - spatial.distance.cosine(list_finale[to_find]['vec'].reshape(-1), xt['vec'])
    cosine_list.append(dict({'res':result, 'i':list_finale[index_image]['index']}))
from operator import itemgetter
cosine_list.sort(key=itemgetter('res'), reverse=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=10,figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    print (cosine_list[indice]['i'])
    row.imshow(array_to_img(list_finale[cosine_list[indice]['i']]['original']))

plt.show()